In [1]:
'''
'''
import warnings

# Ignore warnings
warnings.filterwarnings("ignore")

In [2]:
'''
'''
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Read and standarize the dataset
df = pd.read_csv('data/selected_data.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
X = df.iloc[:,:-1]
y = df.iloc[:,-1]
df = pd.DataFrame(StandardScaler().fit_transform(X), columns=X.columns.values)
df['target'] = y

# Divide dataframe depending on target
df_correct = df.loc[df['target'] == 1]
X_correct = df_correct.iloc[:,:-1]
y_correct = df_correct.iloc[:,-1].reset_index(drop=True)

df_incorrect = df.loc[df['target'] == 0]
X_incorrect = df_incorrect.iloc[:,:-1]
y_incorrect = df_incorrect.iloc[:,-1].reset_index(drop=True)

# Print each target shape
print('Target 1 shape: ' + str(df_correct.shape))
print('Target 0 shape: ' + str(df_incorrect.shape))

Target 1 shape: (5055, 39)
Target 0 shape: (4959, 39)


# PCA

In [3]:
'''
'''
from sklearn.decomposition import PCA
from models.model import ReconstructionErrorModel

# Fit reconstruction error model with correct data
model = ReconstructionErrorModel(X_correct, model=PCA(n_components=0.9))

# Calculate anomaly threshold with correct data
threshold = np.std(model.predict(X_correct)) * 3

# Print principal components
print('%i Principal Components' % len(model.get_model().explained_variance_))

12 Principal Components


In [4]:
'''
'''
# Calculate how many correct instances are detected as anomalies
errors = model.predict(X_correct)
anomalies = np.where(errors > threshold)[0]
print('Anomalies with correct data %s' % str(len(anomalies)))

# Calculate how many correct instances are detected as anomalies
errors = model.predict(X_incorrect)
anomalies = np.where(errors > threshold)[0]
print('Anomalies with incorrect data %s' % str(len(anomalies)))

Anomalies with correct data 84
Anomalies with incorrect data 1117


In [7]:
'''
'''         
from yellowbrick.features import RadViz

# Instantiate the visualizer
visualizer = RadViz()

# Fit the data to the visualizer
data = model.get_model().transform(X_incorrect)
visualizer.fit(data, y_incorrect)           

# Transform the data
visualizer.transform(data)     

# Finalize and render the figure
visualizer.show()    

ValueError: operands could not be broadcast together with shapes (4959,) (1117,) 

# SHAP

In [6]:
'''
'''
import shap

# Summarize training dataset (only instances detected as anomaly)
anomaly_instances = X_incorrect.iloc[anomalies,:].reset_index(drop=True)
train_summary = shap.kmeans(anomaly_instances, 10)

# Calculate anomalies shap values
explainer = shap.KernelExplainer(model.predict, train_summary)
shap_values = explainer.shap_values(anomaly_instances)
df_shap = pd.DataFrame(shap_values, columns=anomaly_instances.columns.values)

# Print global explanation with shap values
shap.summary_plot(df_shap, anomaly_instances, plot_type="bar")

KeyboardInterrupt: 

# Clustering

In [ ]:
'''
'''
from sklearn.cluster import DBSCAN

# Fit DBSCAN clustering algorithm
db_model = DBSCAN(eps=0.5, min_samples=5).fit(df_shap)
labels = db_model.labels_

# Calculate number of clusters and noise points
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

# Print number of clusters and noise points
print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)

# Clusters explainability

In [ ]:
'''
'''
# Make a copy of instances and shap values dataframes
instances = anomaly_instances.copy()
shap_values = df_shap.copy()

# Classify instances and their shap values indicating the cluster number
instances['cluster'] = labels
shap_values['cluster'] = labels

# Generate each clusters explanation
for cluster in np.unique(labels):
    if cluster != -1:
        cluster_instances = instances.loc[instances['cluster'] == cluster].iloc[:,:-1]        
        cluster_shaps = shap_values.loc[shap_values['cluster'] == cluster].iloc[:,:-1]   
        print("CLUSTER {} \nINSTANCES {}".format(cluster, len(cluster_instances)))      
        shap.summary_plot(cluster_shaps, cluster_instances, plot_type="bar")           